In [28]:
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

# nest_asyncio 적용
nest_asyncio.apply()

async def fetch(session, url):
    try:
        async with session.get(url, timeout=aiohttp.ClientTimeout(total=3000)) as response:
            return await response.text()
    except asyncio.TimeoutError:
        print(f"Timeout error for URL: {url}")
        return None

async def get_news_links(session, base_url, page):
    url = f"{base_url}&page={page}"
    response_text = await fetch(session, url)
    if response_text is None:
        return []
    soup = BeautifulSoup(response_text, 'html.parser')
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

async def get_news_content(session, url):
    response_text = await fetch(session, url)
    if response_text is None:
        return None, None
    soup = BeautifulSoup(response_text, 'html.parser')
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article', {'id': 'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    return None, None

async def main():
    today = datetime.today().strftime("%Y%m%d")
    base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today
    async with aiohttp.ClientSession() as session:
        news_links = set()
        page = 1
        
        while True:
            links = await get_news_links(session, base_url, page)
            if not links or page == 500:
                break            
            news_links.update(links)
            page += 1
        

        tasks = [get_news_content(session, link) for link in news_links]
        
        # news_contents = await asyncio.gather(*tasks)
        news_contents = await tqdm_asyncio.gather(*tasks, desc="Fetching news content")
        # None 값을 제거
        news_contents = [content for content in news_contents if content[0] is not None]

        # 명사 추출
        nouns_data = []
        for title, content in news_contents:
            nouns_data.append({'Title': title, 'content': content})
        
        df = pd.DataFrame(nouns_data)
        df.to_csv(f"{today}.csv", index=False, encoding='utf-8-sig')
        # df.to_csv("20240701.csv",index=False,encoding='utf-8-sig')
# Jupyter Notebook 환경에서 실행
await main()


Fetching news content: 100%|██████████| 5050/5050 [12:28<00:00,  6.75it/s]


In [29]:
today = datetime.today().strftime("%Y%m%d")
df = pd.read_csv(f"{today}.csv",encoding='utf-8')
# df = pd.read_csv("20240701.csv",encoding='utf-8')

In [30]:
df

,Title,content
0,"조이, 아이스핀360 신규 컬러 론칭 기념 CJ온스타일 라이브 방송",조이(Joie) 제공(서울=뉴스1) 장도민 기자 = 영국 카시트 브랜드 '조이'(J...
1,"코스닥, 외국인·기관 ‘팔자’…약보합 출발",[이데일리 원다연 기자] 코스닥 지수가 2일 약보합 출발했다. 마켓포인트에 따르면 ...
2,"홈플러스 노조 ""MBK가 산산조각 내""…분할매각 저지 투쟁",홈플러스 직원들이 대주주인 사모펀드(PEF) 운용사 MBK파트너스의 홈플러스 익스프...
3,초소형 온실가스 관측 군집 위성 본격 개발 추진 발표,(세종=뉴스1) 김기남 기자 = 유명수 국립환경과학원 기후대기연구부장이 2일 세종시...
4,"장인화 포스코 회장 ""2030년 시총 200조 목표""",장인화 포스코그룹 회장이 오는 2030년 그룹 시가총액 목표치를 지금의 3배 수준인...
...,...,...
5041,"""일단 긁고 보자"" 카드 할부 벌써 35조…수수료만 '두둑'",무이자 혜택 예전만 못한데지갑 얇아진 서민들만 '씁쓸''최고 19.9%' 수익은 '...
5042,"오메가, 2024 파리 올림픽 타임키핑 신기술 발표",(서울=연합뉴스) 이재희 기자 = 지난 6월 24일 서울 강남구 JW메리어트호텔에서...
5043,엔·달러 환율 161.72엔… 또 37년여만의 최고치,[연합뉴스 제공] 엔·달러 환율이 1일(현지시간) 뉴욕 외환시장에서 한때 달러...
5044,“보험계약전 알릴의무 준수안하면 보험금 제대로 못받아요”,"금감원, 놓치기 쉬운 사항 공개직장인 A 씨는 2021년 건강검진에서 당뇨병이 의심..."
